In [162]:
using GLMakie
using Colors
set_theme!(theme_dark())

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel="Vm (mV)", ylabel="Current (pA)")

sl_1 = Slider(fig[2,1], range = 100:1:160, startvalue = 145)   # Na_out
sl_2 = Slider(fig[3,1], range = 5:0.5:20,   startvalue = 15)   # Na_in
sl_3 = Slider(fig[4,1], range = 2:0.2:10,   startvalue = 5)    # K_out
sl_4 = Slider(fig[5,1], range = 120:1:160,  startvalue = 140)  # K_in

sl_gNa = Slider(fig[6,1], range=0:0.01:1, startvalue=0.2)
sl_gK  = Slider(fig[7,1], range=0:0.1:10, startvalue=1)

lb_1 = Label(fig[2,2], lift(x -> "Na_out = $(x) mM", sl_1.value))
lb_2 = Label(fig[3,2], lift(x -> "Na_in  = $(x) mM", sl_2.value))
lb_3 = Label(fig[4,2], lift(x -> "K_out  = $(x) mM", sl_3.value))
lb_4 = Label(fig[5,2], lift(x -> "K_in   = $(x) mM", sl_4.value))

lb_gNa = Label(fig[6,2], lift(x -> "g_Na  = $(x) nS", sl_gNa.value))
lb_gK = Label(fig[7,2], lift(x -> "g_K   = $(x) nS", sl_gK.value))

Vm = -150:1:150

R = 8.314
T = 310
F = 96_485
z = 1

RTzF = R*T/z/F * 1000 # V -> mV
println("RT/zF=$(round(RTzF, digits=3))")

Ena = lift(sl_1.value, sl_2.value) do Na_out, Na_in
    RTzF * log(Na_out/Na_in)
end
Ek  = lift(sl_3.value, sl_4.value) do K_out, K_in
    RTzF * log(K_out/K_in)
end

I_Na = lift(Ena, sl_gNa.value) do Ena, g_Na
    g_Na.*(Vm .- Ena)
end
I_K  = lift(Ek, sl_gK.value) do Ek, g_K
    g_K.*(Vm .- Ek)
end

ln_Na = lines!(Vm, I_Na, color=RGB(0, 186/255, 255/255))
ln_K  = lines!(Vm, I_K, color=:red)

I_tot = @lift($I_Na .+ $I_K)
ln_tot = lines!(Vm, I_tot, color = :green)

goldman = lift(sl_1.value, 
    sl_2.value, 
    sl_3.value, 
    sl_4.value,
    sl_gNa.value,
    sl_gK.value
) do Na_out, Na_in, K_out, K_in, g_Na, g_K
    RTzF * log(
        (g_Na * Na_out + g_K * K_out) / 
        (g_Na * Na_in  + g_K * K_in)
    )
end

scatter!(@lift([$goldman]), [0], color=:purple)

hodgkin_huxley = lift(Ena, Ek, sl_gNa.value, sl_gK.value) do Ena, Ek, gNa, gK
    (gNa*Ena + gK*Ek) / (gNa + gK)
end
scatter!(@lift([$hodgkin_huxley]), [0], color=:orange)

leg = axislegend(
    ax, 
    [ln_Na, ln_K, ln_tot], 
    ["I_Na", "I_K", "I_tot"],
    position = :rb,
    labelfont = "Georgia"
)

fig

RT/zF=26.712
